In [57]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker, Session
#from sqlalchemy_utils import database_exists, create_database
#from local_settings import postgresql as settings
import pandas as pd
import numpy as np
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine, select, MetaData, Table
from sqlalchemy import Column, String, Integer
import psycopg2

In [58]:
# Creating our engine & session
engine=create_engine('sqlite:///fakedb.sqlite')
session = Session(engine)
Base=automap_base()
Base.prepare(engine, reflect=True)

In [59]:
# Print all of the classes mapped to the Base
Base.classes.keys()

['news', 'trending']

In [60]:
# df_news=pd.read_csv("clean_fakenews.csv")
df_news=pd.read_csv("clean_fakenews_demo.csv")
df_news

,title,text,label
0,law enforcement high alert following threats c...,comment expected barack obama members fyf fuky...,1.0
1,none,post votes hillary already,1.0
2,unbelievable obamas attorney general says char...,demonstrators gathered last night exercising c...,1.0
3,bobby jindal raised hindu uses story christian...,dozen politically active pastors came private ...,0.0
4,satan russia unvelis image terrifying new supe...,rs sarmat missile dubbed satan replace ss flie...,1.0
...,...,...,...
994,south koreas moon unveils new focus southeast ...,bogor indonesia reuters south korean president...,0.0
995,fbi insider clinton emails linked political pe...,posted october sean adltabatabai news us comme...,1.0
996,mainstream media calls real news fake narrativ...,mainstream media calls real news fake narrativ...,1.0
997,new york states challenge trump climate change...,washington reuters coalition us states filed l...,0.0


In [63]:
df_counts=pd.read_csv("counting.csv")
df_counts

,Unnamed: 0,word,total,1_count,0_count
0,0,trump,32668,15615,17053
1,1,people,34088,16291,17797
2,2,president,39887,22555,17332
3,3,clinton,14936,5223,9713
4,4,government,21498,13248,8250
5,5,obama,17310,8232,9078
6,6,campaign,18627,9785,8842
7,7,republican,19817,10978,8839
8,8,election,18758,9894,8864
9,9,hillary,14443,4714,9729


In [64]:

try:
    df_counts.to_sql(name='trending', con=engine)
except:
    print('Check database to see if table already exists')

Check database to see if table already exists


In [65]:
try:
    df_news.to_sql(name='news', con=engine)
except:
    print('Check database to see if table already exists',
         'Success')

Check database to see if table already exists Success


In [66]:
News = Base.classes.news
results = session.query(News.title).all()
results

[('law enforcement high alert following threats cops whites blacklivesmatter fyf terrorists video',),
 ('none',),
 ('unbelievable obamas attorney general says charlotte rioters peaceful protestersin home state north carolina video',),
 ('bobby jindal raised hindu uses story christian conversion woo evangelicals potential bid',),
 ('satan russia unvelis image terrifying new supernuke western world takes notice',),
 ('time christian group sues amazon splc designation hate group',),
 ('dr ben carson targeted irs never audit spoke national prayer breakfast',),
 ('house intel chair trumprussia fake story evidence anything video',),
 ('sports bar owner bans nfl gameswill show true american sports id like speak rural america video',),
 ('latest pipeline leak underscores dangers dakota access pipeline',),
 ('gop senator smacked punchable altright nazi internet',),
 ('may brexit offer would hurt cost eu citizens eu parliament',),
 ('schumer calls trump appoint official oversee puerto rico relie

In [67]:
#Getting the word rows in trending file
Trending = Base.classes.trending
trending = session.query(Trending.word).all()
trending

[('trump',),
 ('people',),
 ('president',),
 ('clinton',),
 ('government',),
 ('obama',),
 ('campaign',),
 ('republican',),
 ('election',),
 ('hillary',)]

In [68]:
session.commit()

In [69]:
Trump=session.query(News.text).filter(News.text.like('%trump'))
for Trumps in Trump:
    print(Trumps)

('share twitter wildfire opinion platform opinions information put forth contributors exclusive represent views ijr panel discussion wednesdays anderso ... (778 characters truncated) ...  video trump brags grabbing women py navarro angrily yelled word cnn panel discussion hughes chastised saying onair well go ahead put ana maybe trump',)


## Trending

>Display trending news and popularity.

>Fetches data from database realtime with most recently added news.

In [70]:
# Get all the trending words over 10,0000

popular=session.query(Trending).filter(Trending.total>=10000)
for words in popular:
    print(words.word+' '.format(words.total))

trump 
people 
president 
clinton 
government 
obama 
campaign 
republican 
election 
hillary 


In [71]:
# Get the total numbers of trending words over 10.000
counts = session.query(Trending).filter(Trending.total >= 10000)
for words in counts:
    print(words.word + ': '+ '{:,}'.format(words.total))

trump: 32,668
people: 34,088
president: 39,887
clinton: 14,936
government: 21,498
obama: 17,310
campaign: 18,627
republican: 19,817
election: 18,758
hillary: 14,443


In [72]:
#  Fetches a random fake news for Trump
import random
trump=session.query(News.title)\
                    .filter(News.title.like('%trump%'))\
                    .filter(News.label == 0)
print(random.choice(list(trump)))

('trump spy chief differ said call russia dossier',)


In [73]:
#Fetches a random fake news for a random trending word
import random
# query for trending words from Trending table
trendy = [
    word[0] for word in session.query(Trending.word).all()
]
print('Top Trending Words: \t' + ' - '.join(trendy))

# choose a random trending word
random_trend = random.choice(trendy)
print('Random Trending Word: \t'+ random_trend)

# 
fetch_news=session.query(News.title)\
                    .filter(News.text.like('%'+ str('mr') +'%'))\
                    .filter(News.label == 0)
print('Random Fake News: \t'+ random.choice(list(fetch_news))[0].title())

Top Trending Words: 	trump - people - president - clinton - government - obama - campaign - republican - election - hillary
Random Trending Word: 	trump
Random Fake News: 	National Biotechnology Panel Faces New Conflict Interest Questions New York Times


In [74]:
populars = session.query(Trending).filter(Trending.fake >= 10000)
print(populars)
print('---------------')
print('Popular Words: ')
for record in populars:
    print('\t'+ record.word + ': '+ '{:,}'.format(record.fake))

SELECT trending.id AS trending_id, trending.word AS trending_word, trending.total AS trending_total, trending.notfake AS trending_notfake, trending.fake AS trending_fake 
FROM trending 
WHERE trending.fake >= ?
---------------
Popular Words: 
	trump: 17,053
	people: 17,797
	president: 17,332
